<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Изучение-и-подготовка-данных" data-toc-modified-id="Изучение-и-подготовка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Изучение и подготовка данных</a></span></li><li><span><a href="#Анализ-различных-моделей-для-определения-стоимости-автомобиля" data-toc-modified-id="Анализ-различных-моделей-для-определения-стоимости-автомобиля-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Анализ различных моделей для определения стоимости автомобиля</a></span><ul class="toc-item"><li><span><a href="#Линейная-регрессия" data-toc-modified-id="Линейная-регрессия-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Линейная регрессия</a></span></li><li><span><a href="#LightGBM" data-toc-modified-id="LightGBM-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>LightGBM</a></span></li><li><span><a href="#XGBoost" data-toc-modified-id="XGBoost-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>XGBoost</a></span></li><li><span><a href="#CatBoost" data-toc-modified-id="CatBoost-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>CatBoost</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li></ul></div>

# Описание проекта
Сервис по продаже автомобилей с пробегом разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В вашем распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. Вам нужно построить модель для определения стоимости.

Заказчику важны:
- качество предсказания;
- скорость предсказания;
- время обучения.

# Инструкция по выполнению проекта:
Чтобы усилить исследование, не ограничивайтесь градиентным бустингом. Попробуйте более простые модели — иногда они работают лучше. Это редкие случаи, которые легко пропустить, если всегда применять только бустинг. Поэкспериментируйте и сравните характеристики моделей: скорость работы, точность результата.
- Загрузите и подготовьте данные.
- Обучите разные модели. Для каждой попробуйте различные гиперпараметры.
- Проанализируйте скорость работы и качество моделей.

Примечания:
- Для оценки качества моделей применяйте метрику RMSE.
- Самостоятельно освойте библиотеку LightGBM и её средствами постройте модели градиентного бустинга.
- Время выполнения ячейки кода Jupyter Notebook можно получить специальной командой. Найдите её.
- Поскольку модель градиентного бустинга может обучаться долго, измените у неё только два-три параметра.
- Если перестанет работать Jupyter Notebook, удалите лишние переменные оператором del.

# Описание данных
- Признаки
    - DateCrawled — дата скачивания анкеты из базы
    - VehicleType — тип автомобильного кузова
    - RegistrationYear — год регистрации автомобиля
    - Gearbox — тип коробки передач
    - Power — мощность (л. с.)
    - Model — модель автомобиля
    - Kilometer — пробег (км)
    - RegistrationMonth — месяц регистрации автомобиля
    - FuelType — тип топлива
    - Brand — марка автомобиля
    - NotRepaired — была машина в ремонте или нет
    - DateCreated — дата создания анкеты
    - NumberOfPictures — количество фотографий автомобиля
    - PostalCode — почтовый индекс владельца анкеты (пользователя)
    - LastSeen — дата последней активности пользователя
- Целевой признак
    - Price — цена (евро)

## Изучение и подготовка данных

In [1]:
import pandas as pd
import numpy as np

from tqdm.auto import tqdm
tqdm.pandas()

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from datetime import datetime
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from lightgbm import LGBMRegressor
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor
from catboost import CatBoostRegressor

In [3]:
# Код для чтения данных
df_autos = pd.read_csv('df_autos.csv')  # Локальный путь

In [4]:
df_autos.head()

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


In [5]:
df_autos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Kilometer          354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  NotRepaired        283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

В данных много пропусков, причем разное количество. Следовательно пропуски не в одних и тех же строках, поэтому рассмотрим каждый столбец с пропусками!

In [6]:
# Найдем дубликаты
df_autos[df_autos.duplicated() == True]

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
171088,2016-03-08 18:42:48,1799,coupe,1999,auto,193,clk,20000,7,petrol,mercedes_benz,no,2016-03-08 00:00:00,0,89518,2016-03-09 09:46:57
231258,2016-03-28 00:56:10,1000,small,2002,manual,83,other,150000,1,petrol,suzuki,no,2016-03-28 00:00:00,0,66589,2016-03-28 08:46:21
258109,2016-04-03 09:01:15,4699,coupe,2003,auto,218,clk,125000,6,petrol,mercedes_benz,yes,2016-04-03 00:00:00,0,75196,2016-04-07 09:44:54
325651,2016-03-18 18:46:15,1999,wagon,2001,manual,131,passat,150000,7,gasoline,volkswagen,no,2016-03-18 00:00:00,0,36391,2016-03-18 18:46:15


In [7]:
# Удалим дубликаты
df_autos = df_autos[df_autos.duplicated() != True]
df_autos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 354365 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354365 non-null  object
 1   Price              354365 non-null  int64 
 2   VehicleType        316875 non-null  object
 3   RegistrationYear   354365 non-null  int64 
 4   Gearbox            334532 non-null  object
 5   Power              354365 non-null  int64 
 6   Model              334660 non-null  object
 7   Kilometer          354365 non-null  int64 
 8   RegistrationMonth  354365 non-null  int64 
 9   FuelType           321470 non-null  object
 10  Brand              354365 non-null  object
 11  NotRepaired        283211 non-null  object
 12  DateCreated        354365 non-null  object
 13  NumberOfPictures   354365 non-null  int64 
 14  PostalCode         354365 non-null  int64 
 15  LastSeen           354365 non-null  object
dtypes: int64(7), object(

Заполним пропуски в колонке с моделью автомобиля, для этого изучим уникальные значения моделей.

In [8]:
df_autos['Model'].unique()

array(['golf', nan, 'grand', 'fabia', '3er', '2_reihe', 'other', 'c_max',
       '3_reihe', 'passat', 'navara', 'ka', 'polo', 'twingo', 'a_klasse',
       'scirocco', '5er', 'meriva', 'arosa', 'c4', 'civic', 'transporter',
       'punto', 'e_klasse', 'clio', 'kadett', 'kangoo', 'corsa', 'one',
       'fortwo', '1er', 'b_klasse', 'signum', 'astra', 'a8', 'jetta',
       'fiesta', 'c_klasse', 'micra', 'vito', 'sprinter', '156', 'escort',
       'forester', 'xc_reihe', 'scenic', 'a4', 'a1', 'insignia', 'combo',
       'focus', 'tt', 'a6', 'jazz', 'omega', 'slk', '7er', '80', '147',
       '100', 'z_reihe', 'sportage', 'sorento', 'v40', 'ibiza', 'mustang',
       'eos', 'touran', 'getz', 'a3', 'almera', 'megane', 'lupo', 'r19',
       'zafira', 'caddy', 'mondeo', 'cordoba', 'colt', 'impreza',
       'vectra', 'berlingo', 'tiguan', 'i_reihe', 'espace', 'sharan',
       '6_reihe', 'panda', 'up', 'seicento', 'ceed', '5_reihe', 'yeti',
       'octavia', 'mii', 'rx_reihe', '6er', 'modus', 'fox'

In [9]:
# Вывод строк с моделью 'other'
df_autos.query('Model == "other"')

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
7,2016-03-21 18:54:38,0,sedan,1980,manual,50,other,40000,7,petrol,volkswagen,no,2016-03-21 00:00:00,0,19348,2016-03-25 16:47:58
24,2016-03-13 20:40:49,500,sedan,1990,manual,118,other,150000,10,petrol,mercedes_benz,yes,2016-03-13 00:00:00,0,35390,2016-03-13 20:40:49
38,2016-04-01 17:45:07,11900,other,2002,manual,129,other,150000,11,gasoline,volkswagen,no,2016-04-01 00:00:00,0,10551,2016-04-05 12:47:30
39,2016-03-25 15:50:30,1500,bus,1984,manual,70,other,150000,8,gasoline,mercedes_benz,no,2016-03-25 00:00:00,0,22767,2016-03-27 03:17:02
52,2016-04-04 10:57:36,1400,NaN,2016,manual,55,other,5000,1,NaN,hyundai,NaN,2016-04-04 00:00:00,0,34454,2016-04-06 12:45:43
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
354290,2016-03-11 14:45:44,2500,small,2007,manual,60,other,125000,3,petrol,hyundai,NaN,2016-03-11 00:00:00,0,18107,2016-03-17 21:46:52
354302,2016-03-12 08:37:15,5990,bus,1984,auto,0,other,70000,4,petrol,chevrolet,NaN,2016-03-12 00:00:00,0,87600,2016-03-24 23:47:48
354316,2016-03-07 19:58:44,3300,coupe,1957,manual,40,other,100000,11,petrol,trabant,no,2016-03-07 00:00:00,0,10317,2016-03-08 06:45:48
354325,2016-03-12 16:48:14,12800,convertible,1991,manual,211,other,150000,7,petrol,porsche,no,2016-03-12 00:00:00,0,21147,2016-04-07 06:46:00


Среди всех моделей есть значение 'other' и оно у разных марок автомобилей, значит все пропуски можно заменить на это значение!

In [10]:
df_autos['Model'] = df_autos['Model'].fillna(value='other')
df_autos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 354365 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354365 non-null  object
 1   Price              354365 non-null  int64 
 2   VehicleType        316875 non-null  object
 3   RegistrationYear   354365 non-null  int64 
 4   Gearbox            334532 non-null  object
 5   Power              354365 non-null  int64 
 6   Model              354365 non-null  object
 7   Kilometer          354365 non-null  int64 
 8   RegistrationMonth  354365 non-null  int64 
 9   FuelType           321470 non-null  object
 10  Brand              354365 non-null  object
 11  NotRepaired        283211 non-null  object
 12  DateCreated        354365 non-null  object
 13  NumberOfPictures   354365 non-null  int64 
 14  PostalCode         354365 non-null  int64 
 15  LastSeen           354365 non-null  object
dtypes: int64(7), object(

Заполним пропуски в колонке с коробкой передач автомобиля, для этого изучим уникальные значения.

In [11]:
df_autos['Gearbox'].unique()

array(['manual', 'auto', nan], dtype=object)

Все пропуски заменим на значение 'indefinite' (неопределенный)

In [12]:
df_autos['Gearbox'] = df_autos['Gearbox'].fillna(value='indefinite')

Заполним пропуски в колонке с типом топлива автомобиля, для этого изучим уникальные значения.

In [13]:
df_autos['FuelType'].unique()

array(['petrol', 'gasoline', nan, 'lpg', 'other', 'hybrid', 'cng',
       'electric'], dtype=object)

Среди значений так же есть 'other', пропуски заменим на это значение.

In [14]:
df_autos['FuelType'] = df_autos['FuelType'].fillna(value='other')

Заполним пропуски в колонке с информацией о ремонте автомобиля, для этого изучим уникальные значения.

In [15]:
df_autos['NotRepaired'].unique()

array([nan, 'yes', 'no'], dtype=object)

In [16]:
df_autos['NotRepaired'].value_counts()

no     247158
yes     36053
Name: NotRepaired, dtype: int64

Будем считать пропуск отсутствием ремонта, поэтому заменим 'nan' на 'no'

In [17]:
df_autos['NotRepaired'] = df_autos['NotRepaired'].fillna(value='no')

Заполним пропуски в колонке с кузовом автомобиля, для этого изучим уникальные значения.

In [18]:
df_autos['VehicleType'].unique()

array([nan, 'coupe', 'suv', 'small', 'sedan', 'convertible', 'bus',
       'wagon', 'other'], dtype=object)

In [19]:
len(df_autos['Model'].unique())

250

In [20]:
len(df_autos['Brand'].unique())

40

Напишем функцию, которая:
- находит все строки с требуемыми маркой и моделью авто
- возвращает самый встречаемый тип кузова среди этих строк

In [21]:
def model_brands_pairs(df_autos=df_autos):
    # Создадим новую таблицу, куда будет записываться модели, марки и наиболее часто встречаемый тип кузова
    df_test = pd.DataFrame(columns=['model', 'brand', 'vehicle_type'])
    
    min_index = 0

    for model in tqdm(df_autos['Model'].unique()):
        for brand in df_autos['Brand'].unique():
            # Найдем все строки с требуемыми маркой и моделью авто
            df_temp = df_autos[(df_autos['Brand'] == brand) & (df_autos['Model'] == model)]
            # Выберем ненулевой длины срезы
            if len(df_temp) == 0:
                continue
            # найдем наиболее часто встречаемый тип кузова при таком сочетании марки и модели
            vehicle_type = df_temp['VehicleType'].value_counts().index[0]
            # в минимальный индекс записываем модель, бренд, тип кузова
            df_test.loc[min_index, 'model'] = model
            df_test.loc[min_index, 'brand'] = brand
            df_test.loc[min_index, 'vehicle_type'] = vehicle_type
            # обновляем индекс
            min_index += 1
    return df_test

In [22]:
try:
    # Откроем таблицу сочетаний модели, марки и  типа кузова
    df_test = pd.read_csv('pair_model_brand.csv')
except FileNotFoundError:
    # если таблицы нет, сгенерируем ее и сохраним
    df_test = model_brands_pairs()
    df_test.to_csv('pair_model_brand.csv', index=False)

In [23]:
df_test.head()

,model,brand,vehicle_type
0,golf,volkswagen,sedan
1,other,volkswagen,small
2,other,audi,wagon
3,other,jeep,suv
4,other,skoda,small


In [24]:
# Заменим пропуски в типе кузова в исходной таблице на наиболее встречаемый при
# определенном сочетании марки и модели
def replace_vehicle_nan(df_autos=df_autos):
    for index in tqdm(df_autos[df_autos['VehicleType'].isna()].index):
        model = df_autos.loc[index, 'Model']
        brand = df_autos.loc[index, 'Brand']
        vehicle_type = df_test[(df_test['brand'] == brand)
                               &(df_test['model'] == model)]['vehicle_type'].values[0]
        df_autos.loc[index, 'VehicleType'] = vehicle_type
    return df_autos

In [25]:
try:
    # Откроем таблицу сочетаний модели, марки и  типа кузова
    df_autos_new = pd.read_csv('df_autos_new.csv')
except FileNotFoundError:
    # если таблицы нет, сгенерируем ее и сохраним
    df_autos_new = replace_vehicle_nan()
    df_autos_new.to_csv('df_autos_new.csv', index=False)

In [26]:
# Найдем минимальную стоимость
df_autos_new['Price'].min()

0

Минимальная стоимость равна нулю, это ошибка, которая внесет в обучение модели неточности, посмотрим какое количество объявлений с нулевой стоимостью

In [27]:
len(df_autos_new.query('Price == 0'))

10772

In [28]:
df_autos_new = df_autos_new.query('Price != 0')

In [29]:
# Найдем количесвто объявлений с единичной стоимостью!
len(df_autos_new.query('Price == 1'))

1189

In [30]:
# Таких объявлений мало? поэтому удалим их
df_autos_new = df_autos_new.query('Price != 1')

In [31]:
df_autos_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 342404 entries, 0 to 354364
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        342404 non-null  object
 1   Price              342404 non-null  int64 
 2   VehicleType        342404 non-null  object
 3   RegistrationYear   342404 non-null  int64 
 4   Gearbox            342404 non-null  object
 5   Power              342404 non-null  int64 
 6   Model              342404 non-null  object
 7   Kilometer          342404 non-null  int64 
 8   RegistrationMonth  342404 non-null  int64 
 9   FuelType           342404 non-null  object
 10  Brand              342404 non-null  object
 11  NotRepaired        342404 non-null  object
 12  DateCreated        342404 non-null  object
 13  NumberOfPictures   342404 non-null  int64 
 14  PostalCode         342404 non-null  int64 
 15  LastSeen           342404 non-null  object
dtypes: int64(7), object(

In [32]:
# Удалим колонки с датой скачивания анкеты, датой создания анкеты и датой последней сессии
df_autos_new = df_autos_new.drop(['DateCrawled', 'DateCreated', 'LastSeen'], axis=1)
df_autos_new.head()

,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,NotRepaired,NumberOfPictures,PostalCode
0,480,sedan,1993,manual,0,golf,150000,0,petrol,volkswagen,no,0,70435
1,18300,coupe,2011,manual,190,other,125000,5,gasoline,audi,yes,0,66954
2,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,no,0,90480
3,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,0,91074
4,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,0,60437


In [33]:
df_autos_new = df_autos_new.drop(['RegistrationMonth'], axis=1)
df_autos_new.head()

,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,FuelType,Brand,NotRepaired,NumberOfPictures,PostalCode
0,480,sedan,1993,manual,0,golf,150000,petrol,volkswagen,no,0,70435
1,18300,coupe,2011,manual,190,other,125000,gasoline,audi,yes,0,66954
2,9800,suv,2004,auto,163,grand,125000,gasoline,jeep,no,0,90480
3,1500,small,2001,manual,75,golf,150000,petrol,volkswagen,no,0,91074
4,3600,small,2008,manual,69,fabia,90000,gasoline,skoda,no,0,60437


Ошибки обработаны, пропуски заполнены, можно переходить к обучению моделей!

In [34]:
# Найдем машины старше 1900 и старше 2021, очевидно, что это ошибки
df_autos_new.query('RegistrationYear < 1900 or RegistrationYear > 2021')

,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,FuelType,Brand,NotRepaired,NumberOfPictures,PostalCode
12946,49,sedan,5000,indefinite,0,golf,5000,other,volkswagen,no,0,74523
15870,1700,sedan,3200,indefinite,0,other,5000,other,sonstige_autos,no,0,33649
16062,190,wagon,1000,indefinite,0,mondeo,5000,other,ford,no,0,47166
17271,700,small,9999,indefinite,0,other,10000,other,opel,no,0,21769
17346,6500,sedan,8888,indefinite,0,other,10000,other,sonstige_autos,no,0,55262
...,...,...,...,...,...,...,...,...,...,...,...,...
331749,99,sedan,9999,indefinite,0,1er,10000,other,bmw,no,0,1239
334963,12000,sedan,4000,indefinite,500,golf,5000,other,volkswagen,no,0,57392
338825,50,sedan,3000,indefinite,3000,golf,100000,other,volkswagen,yes,0,23992
340755,700,sedan,1600,manual,1600,a3,150000,petrol,audi,no,0,86343


In [35]:
# Удалим эти значения
df_autos_new = df_autos_new.query('1900 <= RegistrationYear <= 2021')

In [36]:
# Найдем все авто, у которых мощность больше 500
df_autos_new.query('Power > 500')

,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,FuelType,Brand,NotRepaired,NumberOfPictures,PostalCode
1816,3200,small,2004,manual,1398,corolla,5000,petrol,toyota,no,0,22043
4060,3100,sedan,2005,manual,953,colt,150000,gasoline,mitsubishi,no,0,60326
5328,500,wagon,1999,manual,1001,astra,150000,petrol,opel,no,0,33154
6296,599,small,2002,manual,603,matiz,5000,petrol,chevrolet,yes,0,44379
6504,3000,small,2009,manual,771,punto,125000,petrol,fiat,no,0,40721
...,...,...,...,...,...,...,...,...,...,...,...,...
348964,250,small,1999,manual,1241,ypsilon,150000,petrol,lancia,yes,0,28259
351943,1500,bus,2001,manual,1001,zafira,5000,gasoline,opel,no,0,66117
353489,12500,sedan,2017,manual,2000,other,60000,gasoline,chrysler,no,0,44145
353629,2400,sedan,2007,manual,650,c2,150000,petrol,citroen,no,0,45277


In [37]:
# Всего 417 значений, в основном это ошибки, но ничего страшного если удалим несколько машин
# с реальной такой мощностью
df_autos_new = df_autos_new.query('Power <= 500')

In [38]:
# Найдем количество автомобилей с нулевой мощностью
len(df_autos_new.query('Power == 0'))

35778

In [39]:
df_autos_new.head(10)

,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,FuelType,Brand,NotRepaired,NumberOfPictures,PostalCode
0,480,sedan,1993,manual,0,golf,150000,petrol,volkswagen,no,0,70435
1,18300,coupe,2011,manual,190,other,125000,gasoline,audi,yes,0,66954
2,9800,suv,2004,auto,163,grand,125000,gasoline,jeep,no,0,90480
3,1500,small,2001,manual,75,golf,150000,petrol,volkswagen,no,0,91074
4,3600,small,2008,manual,69,fabia,90000,gasoline,skoda,no,0,60437
5,650,sedan,1995,manual,102,3er,150000,petrol,bmw,yes,0,33775
6,2200,convertible,2004,manual,109,2_reihe,150000,petrol,peugeot,no,0,67112
8,14500,bus,2014,manual,125,c_max,30000,petrol,ford,no,0,94505
9,999,small,1998,manual,101,golf,150000,other,volkswagen,no,0,27472
10,2000,sedan,2004,manual,105,3_reihe,150000,petrol,mazda,no,0,96224


In [40]:
def replace_power_zero(df_autos_new=df_autos_new):
    for row in tqdm(df_autos_new.index):
        if df_autos_new['Power'][row] == 0:
            df_autos_new['Power'][row] = df_autos_new[(df_autos_new['Model'] == df_autos_new['Model'][row])
                                                      & (df_autos_new['Power'] != 0 )]['Power'].mean()
        else:
            continue
    return df_autos_new

In [41]:
try:
    # Откроем таблицу сочетаний модели, марки и  типа кузова
    df_autos_new_01 = pd.read_csv('df_autos_new_01.csv')
except FileNotFoundError:
    # если таблицы нет, сгенерируем ее и сохраним
    df_autos_new_01 = replace_power_zero()
    df_autos_new_01.to_csv('df_autos_new_01.csv', index=False)

In [42]:
df_autos_new_01.head(10)

,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,FuelType,Brand,NotRepaired,NumberOfPictures,PostalCode
0,480,sedan,1993,manual,104.0,golf,150000,petrol,volkswagen,no,0,70435
1,18300,coupe,2011,manual,190.0,other,125000,gasoline,audi,yes,0,66954
2,9800,suv,2004,auto,163.0,grand,125000,gasoline,jeep,no,0,90480
3,1500,small,2001,manual,75.0,golf,150000,petrol,volkswagen,no,0,91074
4,3600,small,2008,manual,69.0,fabia,90000,gasoline,skoda,no,0,60437
5,650,sedan,1995,manual,102.0,3er,150000,petrol,bmw,yes,0,33775
6,2200,convertible,2004,manual,109.0,2_reihe,150000,petrol,peugeot,no,0,67112
7,14500,bus,2014,manual,125.0,c_max,30000,petrol,ford,no,0,94505
8,999,small,1998,manual,101.0,golf,150000,other,volkswagen,no,0,27472
9,2000,sedan,2004,manual,105.0,3_reihe,150000,petrol,mazda,no,0,96224


In [43]:
df_autos_new_01['Power'] = df_autos_new_01['Power'].round()
df_autos_new_01.dropna(subset = ['Power'], inplace = True)
df_autos_new_01.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 341855 entries, 0 to 341855
Data columns (total 12 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Price             341855 non-null  int64  
 1   VehicleType       341855 non-null  object 
 2   RegistrationYear  341855 non-null  int64  
 3   Gearbox           341855 non-null  object 
 4   Power             341855 non-null  float64
 5   Model             341855 non-null  object 
 6   Kilometer         341855 non-null  int64  
 7   FuelType          341855 non-null  object 
 8   Brand             341855 non-null  object 
 9   NotRepaired       341855 non-null  object 
 10  NumberOfPictures  341855 non-null  int64  
 11  PostalCode        341855 non-null  int64  
dtypes: float64(1), int64(5), object(6)
memory usage: 33.9+ MB


In [44]:
df_autos_new_01['Power'] = df_autos_new_01['Power'].astype('int')

## Анализ различных моделей для определения стоимости автомобиля

In [45]:
# Выполним прямое кодирование таблицы с обходом дамми-ловушки
df_autos_new_ohe = pd.get_dummies(df_autos_new_01, drop_first=True)
df_autos_new_ohe

,Price,RegistrationYear,Power,Kilometer,NumberOfPictures,PostalCode,VehicleType_convertible,VehicleType_coupe,VehicleType_other,VehicleType_sedan,...,Brand_skoda,Brand_smart,Brand_sonstige_autos,Brand_subaru,Brand_suzuki,Brand_toyota,Brand_trabant,Brand_volkswagen,Brand_volvo,NotRepaired_yes
0,480,1993,104,150000,0,70435,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
1,18300,2011,190,125000,0,66954,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
2,9800,2004,163,125000,0,90480,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1500,2001,75,150000,0,91074,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,3600,2008,69,90000,0,60437,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
341851,1150,2000,125,150000,0,26624,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
341852,2200,2005,125,20000,0,39576,0,0,0,1,...,0,0,1,0,0,0,0,0,0,0
341853,1199,2000,101,125000,0,26135,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0
341854,9200,1996,102,150000,0,87439,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [46]:
# Выделим переменные признаки и признак, который нужно предсказать для каждой таблицы
features = df_autos_new_ohe.drop('Price', axis=1)
target = df_autos_new_ohe['Price']
# Отделим валидационную выборку (25%) от первоначальных данных
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.25, random_state=12345)

### Линейная регрессия

In [47]:
start_time = datetime.now()

model_lin_reg = LinearRegression()
result_lin_reg = cross_val_score(model_lin_reg, features_train, target_train,
                                 scoring='neg_mean_squared_error', cv=5).mean()
end_time = datetime.now()
time_lin_reg = end_time - start_time
print('RMSE модели линейной регрессии:', round(abs(result_lin_reg)**0.5, 2))
print('Время выполнения кода:', time_lin_reg)

RMSE модели линейной регрессии: 2752.61
Время выполнения кода: 0:00:10.579188


In [48]:
# Найдем время обучения и предсказания модели линейной регрессии
start_time = datetime.now()
model_lin_reg.fit(features_train, target_train)
end_time = datetime.now()
time_lr_fit = end_time - start_time

start_time = datetime.now()
predictions=model_lin_reg.predict(features_test)
end_time = datetime.now()
time_lr_predict = end_time - start_time
print('Время обучения модели линейной регрессии:', time_lr_fit)
print('Время предсказания модели линейной регрессии:', time_lr_predict)

Время обучения модели линейной регрессии: 0:00:02.601249
Время предсказания модели линейной регрессии: 0:00:00.145788


### LightGBM

In [49]:
df_autos_new_01.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 341855 entries, 0 to 341855
Data columns (total 12 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   Price             341855 non-null  int64 
 1   VehicleType       341855 non-null  object
 2   RegistrationYear  341855 non-null  int64 
 3   Gearbox           341855 non-null  object
 4   Power             341855 non-null  int64 
 5   Model             341855 non-null  object
 6   Kilometer         341855 non-null  int64 
 7   FuelType          341855 non-null  object
 8   Brand             341855 non-null  object
 9   NotRepaired       341855 non-null  object
 10  NumberOfPictures  341855 non-null  int64 
 11  PostalCode        341855 non-null  int64 
dtypes: int64(6), object(6)
memory usage: 33.9+ MB


In [50]:
# Поменяем тип данных на категориальный в следующих стобцах
for col in ['VehicleType', 'Gearbox', 'Model', 'FuelType', 'Brand', 'NotRepaired']:
    df_autos_new_01[col] = df_autos_new_01[col].astype('category')

In [51]:
df_autos_new_01.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 341855 entries, 0 to 341855
Data columns (total 12 columns):
 #   Column            Non-Null Count   Dtype   
---  ------            --------------   -----   
 0   Price             341855 non-null  int64   
 1   VehicleType       341855 non-null  category
 2   RegistrationYear  341855 non-null  int64   
 3   Gearbox           341855 non-null  category
 4   Power             341855 non-null  int64   
 5   Model             341855 non-null  category
 6   Kilometer         341855 non-null  int64   
 7   FuelType          341855 non-null  category
 8   Brand             341855 non-null  category
 9   NotRepaired       341855 non-null  category
 10  NumberOfPictures  341855 non-null  int64   
 11  PostalCode        341855 non-null  int64   
dtypes: category(6), int64(6)
memory usage: 20.6 MB


In [54]:
# Выделим переменные признаки и признак, который нужно предсказать для каждой таблицы
features_categorial = df_autos_new_01.drop('Price', axis=1)
target_categorial = df_autos_new_01['Price']
# Отделим валидационную выборку (20%) от первоначальных данных
features_train_categorial, features_test_categorial, target_train_categorial, target_test_categorial = train_test_split(
    features_categorial, target_categorial, test_size=0.20, random_state=12345)

In [55]:
start_time = datetime.now()

model_lgbm = LGBMRegressor(random_state=12345)
params_lgbm = {'n_estimators' : [100, 200, 500],
               'max_depth' : range(2, 7),
               'num_leaves': [20, 30, 40],
               'learning_rate': [0.03, 0.1, 0.5]}

search_lgbm = GridSearchCV(estimator=model_lgbm, param_grid=params_lgbm, scoring='neg_mean_squared_error',
                           verbose=3, cv=3)
result_lgbm = search_lgbm.fit(features_train_categorial , target_train_categorial)

end_time = datetime.now()
time_lgbm = end_time - start_time

Fitting 3 folds for each of 135 candidates, totalling 405 fits
[CV 1/3] END learning_rate=0.03, max_depth=2, n_estimators=100, num_leaves=20;, score=-6253206.365 total time=   0.3s
[CV 2/3] END learning_rate=0.03, max_depth=2, n_estimators=100, num_leaves=20;, score=-6181485.762 total time=   0.3s
[CV 3/3] END learning_rate=0.03, max_depth=2, n_estimators=100, num_leaves=20;, score=-6155917.584 total time=   0.3s
[CV 1/3] END learning_rate=0.03, max_depth=2, n_estimators=100, num_leaves=30;, score=-6253206.365 total time=   0.3s
[CV 2/3] END learning_rate=0.03, max_depth=2, n_estimators=100, num_leaves=30;, score=-6181485.762 total time=   0.3s
[CV 3/3] END learning_rate=0.03, max_depth=2, n_estimators=100, num_leaves=30;, score=-6155917.584 total time=   0.4s
[CV 1/3] END learning_rate=0.03, max_depth=2, n_estimators=100, num_leaves=40;, score=-6253206.365 total time=   0.3s
[CV 2/3] END learning_rate=0.03, max_depth=2, n_estimators=100, num_leaves=40;, score=-6181485.762 total time= 

[CV 1/3] END learning_rate=0.03, max_depth=4, n_estimators=200, num_leaves=40;, score=-3346585.236 total time=   0.7s
[CV 2/3] END learning_rate=0.03, max_depth=4, n_estimators=200, num_leaves=40;, score=-3366359.168 total time=   0.7s
[CV 3/3] END learning_rate=0.03, max_depth=4, n_estimators=200, num_leaves=40;, score=-3363644.976 total time=   0.7s
[CV 1/3] END learning_rate=0.03, max_depth=4, n_estimators=500, num_leaves=20;, score=-3032008.818 total time=   1.5s
[CV 2/3] END learning_rate=0.03, max_depth=4, n_estimators=500, num_leaves=20;, score=-3050104.711 total time=   1.5s
[CV 3/3] END learning_rate=0.03, max_depth=4, n_estimators=500, num_leaves=20;, score=-3041832.940 total time=   1.5s
[CV 1/3] END learning_rate=0.03, max_depth=4, n_estimators=500, num_leaves=30;, score=-3032008.818 total time=   1.5s
[CV 2/3] END learning_rate=0.03, max_depth=4, n_estimators=500, num_leaves=30;, score=-3050104.711 total time=   1.5s
[CV 3/3] END learning_rate=0.03, max_depth=4, n_estimato

[CV 2/3] END learning_rate=0.1, max_depth=2, n_estimators=100, num_leaves=30;, score=-4087827.147 total time=   0.3s
[CV 3/3] END learning_rate=0.1, max_depth=2, n_estimators=100, num_leaves=30;, score=-4093464.858 total time=   0.3s
[CV 1/3] END learning_rate=0.1, max_depth=2, n_estimators=100, num_leaves=40;, score=-4064992.995 total time=   0.3s
[CV 2/3] END learning_rate=0.1, max_depth=2, n_estimators=100, num_leaves=40;, score=-4087827.147 total time=   0.3s
[CV 3/3] END learning_rate=0.1, max_depth=2, n_estimators=100, num_leaves=40;, score=-4093464.858 total time=   0.3s
[CV 1/3] END learning_rate=0.1, max_depth=2, n_estimators=200, num_leaves=20;, score=-3687127.110 total time=   0.5s
[CV 2/3] END learning_rate=0.1, max_depth=2, n_estimators=200, num_leaves=20;, score=-3715787.625 total time=   0.5s
[CV 3/3] END learning_rate=0.1, max_depth=2, n_estimators=200, num_leaves=20;, score=-3678724.371 total time=   0.5s
[CV 1/3] END learning_rate=0.1, max_depth=2, n_estimators=200, n

[CV 1/3] END learning_rate=0.1, max_depth=4, n_estimators=500, num_leaves=30;, score=-2775450.352 total time=   1.8s
[CV 2/3] END learning_rate=0.1, max_depth=4, n_estimators=500, num_leaves=30;, score=-2802187.212 total time=   1.8s
[CV 3/3] END learning_rate=0.1, max_depth=4, n_estimators=500, num_leaves=30;, score=-2802288.085 total time=   1.8s
[CV 1/3] END learning_rate=0.1, max_depth=4, n_estimators=500, num_leaves=40;, score=-2775450.352 total time=   1.8s
[CV 2/3] END learning_rate=0.1, max_depth=4, n_estimators=500, num_leaves=40;, score=-2802187.212 total time=   1.5s
[CV 3/3] END learning_rate=0.1, max_depth=4, n_estimators=500, num_leaves=40;, score=-2802288.085 total time=   1.5s
[CV 1/3] END learning_rate=0.1, max_depth=5, n_estimators=100, num_leaves=20;, score=-3016529.292 total time=   0.6s
[CV 2/3] END learning_rate=0.1, max_depth=5, n_estimators=100, num_leaves=20;, score=-3036898.057 total time=   0.5s
[CV 3/3] END learning_rate=0.1, max_depth=5, n_estimators=100, n

[CV 3/3] END learning_rate=0.5, max_depth=2, n_estimators=200, num_leaves=20;, score=-3283610.220 total time=   0.5s
[CV 1/3] END learning_rate=0.5, max_depth=2, n_estimators=200, num_leaves=30;, score=-3210999.800 total time=   0.5s
[CV 2/3] END learning_rate=0.5, max_depth=2, n_estimators=200, num_leaves=30;, score=-3234970.307 total time=   0.5s
[CV 3/3] END learning_rate=0.5, max_depth=2, n_estimators=200, num_leaves=30;, score=-3283610.220 total time=   0.5s
[CV 1/3] END learning_rate=0.5, max_depth=2, n_estimators=200, num_leaves=40;, score=-3210999.800 total time=   0.5s
[CV 2/3] END learning_rate=0.5, max_depth=2, n_estimators=200, num_leaves=40;, score=-3234970.307 total time=   0.5s
[CV 3/3] END learning_rate=0.5, max_depth=2, n_estimators=200, num_leaves=40;, score=-3283610.220 total time=   0.5s
[CV 1/3] END learning_rate=0.5, max_depth=2, n_estimators=500, num_leaves=20;, score=-3008304.457 total time=   0.9s
[CV 2/3] END learning_rate=0.5, max_depth=2, n_estimators=500, n

[CV 2/3] END learning_rate=0.5, max_depth=5, n_estimators=100, num_leaves=20;, score=-2914709.978 total time=   0.4s
[CV 3/3] END learning_rate=0.5, max_depth=5, n_estimators=100, num_leaves=20;, score=-2920363.356 total time=   0.4s
[CV 1/3] END learning_rate=0.5, max_depth=5, n_estimators=100, num_leaves=30;, score=-2854058.348 total time=   0.5s
[CV 2/3] END learning_rate=0.5, max_depth=5, n_estimators=100, num_leaves=30;, score=-2894135.664 total time=   0.5s
[CV 3/3] END learning_rate=0.5, max_depth=5, n_estimators=100, num_leaves=30;, score=-2864353.256 total time=   0.5s
[CV 1/3] END learning_rate=0.5, max_depth=5, n_estimators=100, num_leaves=40;, score=-2852963.434 total time=   0.5s
[CV 2/3] END learning_rate=0.5, max_depth=5, n_estimators=100, num_leaves=40;, score=-2908322.466 total time=   0.5s
[CV 3/3] END learning_rate=0.5, max_depth=5, n_estimators=100, num_leaves=40;, score=-2864366.523 total time=   0.5s
[CV 1/3] END learning_rate=0.5, max_depth=5, n_estimators=200, n

In [58]:
print('Лучшее значение RMSE в LightGBM:', round(abs(result_lgbm.best_score_)**0.5, 2))
print('Лучшие параметры LightGBM:', result_lgbm.best_params_)
print('Время выполнения кода:', time_lgbm)

Лучшее значение RMSE в LightGBM: 1621.63
Лучшие параметры LightGBM: {'learning_rate': 0.1, 'max_depth': 6, 'n_estimators': 500, 'num_leaves': 40}
Время выполнения кода: 0:06:03.041867


Мы перебрали много параметров, каждый щаг считался достаточно быстро: 1-2 секунды, в общем выполнение кода заняло 10 минут, это хорошее время для такого количество шагов, RMSE = 1621, это почти в два раза лучше результата в линейной регрессии.

In [60]:
# Найдем время обучения и предсказания модели LightGBM
start_time = datetime.now()
best_model_lgbm = search_lgbm.best_estimator_
best_model_lgbm.fit(features_train_categorial, target_train_categorial)
end_time = datetime.now()
time_lgbm_fit = end_time - start_time

start_time = datetime.now()
predictions=best_model_lgbm.predict(features_test_categorial)
end_time = datetime.now()
time_lgbm_predict = end_time - start_time
print('Время обучения модели LightGBM:', time_lgbm_fit)
print('Время предсказания модели LightGBM:', time_lgbm_predict)

Время обучения модели LightGBM: 0:00:02.104028
Время предсказания модели LightGBM: 0:00:00.335054


### XGBoost

In [61]:
start_time = datetime.now()

model_xgb = XGBRegressor(random_state=12345)
params_xgb = {'eta' : [0.1, 0.3, 0.5],
               'max_depth' : [6]}

search_xgb = GridSearchCV(estimator=model_xgb, param_grid=params_xgb, scoring='neg_mean_squared_error',
                           verbose=5, cv=3)
result_xgb = search_xgb.fit(features_train , target_train)

end_time = datetime.now()
time_xgb = end_time - start_time

Fitting 3 folds for each of 3 candidates, totalling 9 fits
[CV 1/3] END .......eta=0.1, max_depth=6;, score=-3095642.746 total time=  47.0s
[CV 2/3] END .......eta=0.1, max_depth=6;, score=-3100871.004 total time=  51.4s
[CV 3/3] END .......eta=0.1, max_depth=6;, score=-3161173.403 total time=  52.4s
[CV 1/3] END .......eta=0.3, max_depth=6;, score=-2914307.738 total time=  52.6s
[CV 2/3] END .......eta=0.3, max_depth=6;, score=-2933719.918 total time=  52.8s
[CV 3/3] END .......eta=0.3, max_depth=6;, score=-2934120.638 total time=  51.8s
[CV 1/3] END .......eta=0.5, max_depth=6;, score=-2896571.330 total time=  52.4s
[CV 2/3] END .......eta=0.5, max_depth=6;, score=-2900111.531 total time=  53.3s
[CV 3/3] END .......eta=0.5, max_depth=6;, score=-2914440.172 total time=  54.4s


In [62]:
print('Лучшее значение RMSE в XGBoost:', round(abs(result_xgb.best_score_)**0.5, 2))
print('Лучшие параметры XGBoost:', result_xgb.best_params_)
print('Время выполнения кода:', time_xgb)

Лучшее значение RMSE в XGBoost: 1704.03
Лучшие параметры XGBoost: {'eta': 0.5, 'max_depth': 6}
Время выполнения кода: 0:09:08.699186


Сделали всего 9 шагов, и каждый шаг занял около 53 секунд, это намного дольше чем в LightGBM, однако RMSE = 1704, что близко к результату lightGBM, но все же чуть больше.

In [76]:
# Найдем время обучения и предсказания модели XGBoost
start_time = datetime.now()
best_model_xbg = search_xgb.best_estimator_
best_model_xbg.fit(features_train, target_train)
end_time = datetime.now()
time_xgb_fit = end_time - start_time

start_time = datetime.now()
predictions=best_model_xbg.predict(features_test)
end_time = datetime.now()
time_xgb_predict = end_time - start_time
print('Время обучения модели XGBoost:', time_xgb_fit)
print('Время предсказания модели XGBoost:', time_xgb_predict)

Время обучения модели XGBoost: 0:01:08.542189
Время предсказания модели XGBoost: 0:00:00.261927


### CatBoost

In [66]:
start_time = datetime.now()

model_cat = CatBoostRegressor(random_seed=12345)
params_cat = {'iterations' : [100, 300, 500],
              'depth' : range(4, 8),
              'grow_policy' : ['SymmetricTree']}

search_cat = GridSearchCV(estimator=model_cat, param_grid=params_cat, scoring='neg_mean_squared_error',
                           verbose=1, cv=3)
result_cat = search_cat.fit(features_train, target_train)

end_time = datetime.now()
time_cat = end_time - start_time

Fitting 3 folds for each of 12 candidates, totalling 36 fits
Learning rate set to 0.5
0:	learn: 3440.5774848	total: 69.7ms	remaining: 6.89s
1:	learn: 2844.9246012	total: 76.2ms	remaining: 3.73s
2:	learn: 2576.5544905	total: 81.4ms	remaining: 2.63s
3:	learn: 2436.5221506	total: 88.2ms	remaining: 2.12s
4:	learn: 2333.1243026	total: 94.4ms	remaining: 1.79s
5:	learn: 2275.8465628	total: 101ms	remaining: 1.58s
6:	learn: 2228.9726571	total: 107ms	remaining: 1.42s
7:	learn: 2198.6819355	total: 113ms	remaining: 1.3s
8:	learn: 2166.5171045	total: 120ms	remaining: 1.21s
9:	learn: 2144.2111543	total: 126ms	remaining: 1.14s
10:	learn: 2121.6614146	total: 133ms	remaining: 1.08s
11:	learn: 2104.7949414	total: 139ms	remaining: 1.02s
12:	learn: 2085.3522676	total: 146ms	remaining: 975ms
13:	learn: 2071.1068809	total: 152ms	remaining: 932ms
14:	learn: 2060.8471252	total: 158ms	remaining: 896ms
15:	learn: 2047.2850893	total: 166ms	remaining: 870ms
16:	learn: 2040.2451523	total: 172ms	remaining: 842ms
17

54:	learn: 1870.1565803	total: 365ms	remaining: 299ms
55:	learn: 1868.1433421	total: 373ms	remaining: 293ms
56:	learn: 1865.2717731	total: 381ms	remaining: 287ms
57:	learn: 1862.3574134	total: 388ms	remaining: 281ms
58:	learn: 1860.8123338	total: 394ms	remaining: 274ms
59:	learn: 1858.9633183	total: 402ms	remaining: 268ms
60:	learn: 1856.0396346	total: 409ms	remaining: 261ms
61:	learn: 1853.8556123	total: 416ms	remaining: 255ms
62:	learn: 1851.9447916	total: 422ms	remaining: 248ms
63:	learn: 1850.0309763	total: 429ms	remaining: 241ms
64:	learn: 1846.3095489	total: 436ms	remaining: 235ms
65:	learn: 1843.6805641	total: 442ms	remaining: 228ms
66:	learn: 1841.4108696	total: 448ms	remaining: 221ms
67:	learn: 1839.6475674	total: 454ms	remaining: 214ms
68:	learn: 1836.2791890	total: 461ms	remaining: 207ms
69:	learn: 1834.2232005	total: 467ms	remaining: 200ms
70:	learn: 1832.0162287	total: 473ms	remaining: 193ms
71:	learn: 1828.4517313	total: 480ms	remaining: 187ms
72:	learn: 1826.8378479	tota

21:	learn: 2122.3149933	total: 163ms	remaining: 2.06s
22:	learn: 2112.3671481	total: 170ms	remaining: 2.05s
23:	learn: 2104.6201410	total: 177ms	remaining: 2.04s
24:	learn: 2096.1966425	total: 184ms	remaining: 2.02s
25:	learn: 2088.9354818	total: 190ms	remaining: 2s
26:	learn: 2078.3501527	total: 197ms	remaining: 1.99s
27:	learn: 2071.1675000	total: 203ms	remaining: 1.97s
28:	learn: 2063.0064323	total: 209ms	remaining: 1.96s
29:	learn: 2054.5066662	total: 216ms	remaining: 1.95s
30:	learn: 2049.2070154	total: 222ms	remaining: 1.93s
31:	learn: 2041.3159723	total: 229ms	remaining: 1.92s
32:	learn: 2033.0237047	total: 235ms	remaining: 1.91s
33:	learn: 2028.2307470	total: 242ms	remaining: 1.89s
34:	learn: 2024.3096918	total: 248ms	remaining: 1.88s
35:	learn: 2016.4475373	total: 254ms	remaining: 1.86s
36:	learn: 2009.3309098	total: 261ms	remaining: 1.85s
37:	learn: 2004.1088317	total: 267ms	remaining: 1.84s
38:	learn: 1998.5340115	total: 273ms	remaining: 1.83s
39:	learn: 1995.0253821	total: 

172:	learn: 1784.6854924	total: 1.15s	remaining: 846ms
173:	learn: 1783.9859172	total: 1.16s	remaining: 841ms
174:	learn: 1783.4259378	total: 1.17s	remaining: 835ms
175:	learn: 1783.1209078	total: 1.18s	remaining: 830ms
176:	learn: 1782.3237163	total: 1.19s	remaining: 825ms
177:	learn: 1781.6380875	total: 1.2s	remaining: 820ms
178:	learn: 1781.0037546	total: 1.21s	remaining: 816ms
179:	learn: 1780.1034142	total: 1.22s	remaining: 811ms
180:	learn: 1779.4651257	total: 1.23s	remaining: 808ms
181:	learn: 1778.7618886	total: 1.25s	remaining: 812ms
182:	learn: 1777.7408214	total: 1.27s	remaining: 815ms
183:	learn: 1776.3890109	total: 1.28s	remaining: 807ms
184:	learn: 1775.9017926	total: 1.29s	remaining: 800ms
185:	learn: 1775.4483734	total: 1.29s	remaining: 793ms
186:	learn: 1774.9094093	total: 1.3s	remaining: 785ms
187:	learn: 1774.3435804	total: 1.3s	remaining: 778ms
188:	learn: 1773.7670354	total: 1.31s	remaining: 770ms
189:	learn: 1772.8520317	total: 1.32s	remaining: 763ms
190:	learn: 1

25:	learn: 2083.4366210	total: 168ms	remaining: 1.77s
26:	learn: 2072.0405813	total: 174ms	remaining: 1.76s
27:	learn: 2064.0282416	total: 182ms	remaining: 1.76s
28:	learn: 2056.4558222	total: 188ms	remaining: 1.75s
29:	learn: 2049.0610118	total: 194ms	remaining: 1.75s
30:	learn: 2043.0038576	total: 200ms	remaining: 1.74s
31:	learn: 2037.9096087	total: 207ms	remaining: 1.73s
32:	learn: 2033.2927835	total: 213ms	remaining: 1.72s
33:	learn: 2023.6718922	total: 220ms	remaining: 1.72s
34:	learn: 2019.3899960	total: 226ms	remaining: 1.71s
35:	learn: 2015.3862226	total: 232ms	remaining: 1.7s
36:	learn: 2007.6836356	total: 238ms	remaining: 1.69s
37:	learn: 2001.9756165	total: 244ms	remaining: 1.68s
38:	learn: 1996.2322493	total: 251ms	remaining: 1.68s
39:	learn: 1991.7527300	total: 257ms	remaining: 1.67s
40:	learn: 1987.5560077	total: 263ms	remaining: 1.66s
41:	learn: 1982.7144546	total: 270ms	remaining: 1.66s
42:	learn: 1979.1584219	total: 276ms	remaining: 1.65s
43:	learn: 1976.6710338	total

...

231:	learn: 1660.4675331	total: 1.98s	remaining: 2.29s
232:	learn: 1659.9887731	total: 1.99s	remaining: 2.28s
233:	learn: 1659.3044288	total: 2s	remaining: 2.27s
234:	learn: 1658.9860951	total: 2.01s	remaining: 2.26s
235:	learn: 1658.5240992	total: 2.02s	remaining: 2.25s
236:	learn: 1658.1242228	total: 2.02s	remaining: 2.25s
237:	learn: 1657.5062459	total: 2.03s	remaining: 2.24s
238:	learn: 1656.8084871	total: 2.04s	remaining: 2.23s
239:	learn: 1656.2920098	total: 2.05s	remaining: 2.22s
240:	learn: 1656.1091028	total: 2.06s	remaining: 2.21s
241:	learn: 1655.6090595	total: 2.06s	remaining: 2.2s
242:	learn: 1655.0027743	total: 2.07s	remaining: 2.19s
243:	learn: 1654.4253742	total: 2.08s	remaining: 2.19s
244:	learn: 1653.8937284	total: 2.09s	remaining: 2.18s
245:	learn: 1653.5537301	total: 2.1s	remaining: 2.17s
246:	learn: 1653.2014295	total: 2.11s	remaining: 2.16s
247:	learn: 1652.7240945	total: 2.11s	remaining: 2.15s
248:	learn: 1652.0503358	total: 2.12s	remaining: 2.14s
249:	learn: 165

398:	learn: 1591.8633988	total: 3.39s	remaining: 859ms
399:	learn: 1591.5837721	total: 3.4s	remaining: 851ms
400:	learn: 1591.2559988	total: 3.41s	remaining: 842ms
401:	learn: 1590.9492265	total: 3.42s	remaining: 833ms
402:	learn: 1590.5396694	total: 3.43s	remaining: 825ms
403:	learn: 1590.2115748	total: 3.44s	remaining: 816ms
404:	learn: 1589.8504699	total: 3.44s	remaining: 808ms
405:	learn: 1589.4391785	total: 3.45s	remaining: 799ms
406:	learn: 1589.1067805	total: 3.46s	remaining: 791ms
407:	learn: 1588.7490932	total: 3.47s	remaining: 782ms
408:	learn: 1588.5051574	total: 3.48s	remaining: 774ms
409:	learn: 1588.3648372	total: 3.48s	remaining: 765ms
410:	learn: 1588.2495427	total: 3.49s	remaining: 757ms
411:	learn: 1588.0263412	total: 3.5s	remaining: 748ms
412:	learn: 1587.7548425	total: 3.51s	remaining: 740ms
413:	learn: 1587.5152152	total: 3.52s	remaining: 731ms
414:	learn: 1587.1904615	total: 3.53s	remaining: 723ms
415:	learn: 1586.7528359	total: 3.54s	remaining: 714ms
416:	learn: 

49:	learn: 1855.1503161	total: 555ms	remaining: 5s
50:	learn: 1852.4354275	total: 569ms	remaining: 5.01s
51:	learn: 1850.3572751	total: 580ms	remaining: 4.99s
52:	learn: 1847.7228111	total: 591ms	remaining: 4.98s
53:	learn: 1845.1688809	total: 602ms	remaining: 4.97s
54:	learn: 1842.4397099	total: 612ms	remaining: 4.95s
55:	learn: 1839.5348112	total: 623ms	remaining: 4.94s
56:	learn: 1837.7981018	total: 633ms	remaining: 4.92s
57:	learn: 1834.9610023	total: 645ms	remaining: 4.91s
58:	learn: 1832.8901465	total: 656ms	remaining: 4.9s
59:	learn: 1831.1200563	total: 666ms	remaining: 4.89s
60:	learn: 1828.7760403	total: 677ms	remaining: 4.87s
61:	learn: 1827.2502832	total: 687ms	remaining: 4.85s
62:	learn: 1823.9850458	total: 698ms	remaining: 4.84s
63:	learn: 1821.8279845	total: 709ms	remaining: 4.83s
64:	learn: 1819.4682954	total: 720ms	remaining: 4.82s
65:	learn: 1817.8557412	total: 731ms	remaining: 4.81s
66:	learn: 1816.7908877	total: 741ms	remaining: 4.79s
67:	learn: 1815.4303014	total: 7

201:	learn: 1678.4283097	total: 2.21s	remaining: 3.25s
202:	learn: 1677.9713810	total: 2.22s	remaining: 3.24s
203:	learn: 1677.4996370	total: 2.23s	remaining: 3.23s
204:	learn: 1676.9497714	total: 2.24s	remaining: 3.22s
205:	learn: 1676.5027813	total: 2.25s	remaining: 3.21s
206:	learn: 1675.9934479	total: 2.26s	remaining: 3.2s
207:	learn: 1675.6548791	total: 2.27s	remaining: 3.19s
208:	learn: 1675.1459041	total: 2.28s	remaining: 3.17s
209:	learn: 1674.6990756	total: 2.29s	remaining: 3.16s
210:	learn: 1674.1334677	total: 2.3s	remaining: 3.15s
211:	learn: 1673.8136919	total: 2.31s	remaining: 3.14s
212:	learn: 1673.2062415	total: 2.32s	remaining: 3.13s
213:	learn: 1672.4432618	total: 2.33s	remaining: 3.12s
214:	learn: 1671.8661057	total: 2.34s	remaining: 3.11s
215:	learn: 1671.0911141	total: 2.35s	remaining: 3.1s
216:	learn: 1670.5344741	total: 2.37s	remaining: 3.08s
217:	learn: 1669.9665117	total: 2.38s	remaining: 3.07s
218:	learn: 1669.4819659	total: 2.39s	remaining: 3.06s
219:	learn: 1

353:	learn: 1614.5963650	total: 3.84s	remaining: 1.58s
354:	learn: 1614.3149483	total: 3.86s	remaining: 1.57s
355:	learn: 1614.0144598	total: 3.87s	remaining: 1.56s
356:	learn: 1613.6154710	total: 3.88s	remaining: 1.55s
357:	learn: 1613.3800620	total: 3.89s	remaining: 1.54s
358:	learn: 1612.9929903	total: 3.9s	remaining: 1.53s
359:	learn: 1612.7339536	total: 3.91s	remaining: 1.52s
360:	learn: 1612.4781527	total: 3.92s	remaining: 1.51s
361:	learn: 1612.0618188	total: 3.93s	remaining: 1.5s
362:	learn: 1611.5115512	total: 3.94s	remaining: 1.49s
363:	learn: 1611.2745059	total: 3.95s	remaining: 1.48s
364:	learn: 1611.0116554	total: 3.96s	remaining: 1.47s
365:	learn: 1610.9592780	total: 3.97s	remaining: 1.45s
366:	learn: 1610.5564803	total: 3.98s	remaining: 1.44s
367:	learn: 1610.3603827	total: 3.99s	remaining: 1.43s
368:	learn: 1610.0382323	total: 4s	remaining: 1.42s
369:	learn: 1609.7712076	total: 4.02s	remaining: 1.41s
370:	learn: 1609.4520927	total: 4.03s	remaining: 1.4s
371:	learn: 1609

In [67]:
print('Лучшее значение RMSE в CatBoost:', round(abs(result_cat.best_score_)**0.5, 2))
print('Лучшие параметры CatBoost:', result_cat.best_params_)
print('Время выполнения кода:', time_cat)

Лучшее значение RMSE в CatBoost: 1659.83
Лучшие параметры CatBoost: {'depth': 7, 'grow_policy': 'SymmetricTree', 'iterations': 500}
Время выполнения кода: 0:01:38.500856


lightGBM выполнила 405 операций за 6 минут, это примерно 68 операций в минуту, CatBoost выполнила 36 операций за 1 минуту 40 сенкунд, это примерно 20 операций в минуту. CatBoost заметно дольше, даже с параметром grow_policy = SymmetricTree. RMSE этих моделей при этом примерно равны.

In [68]:
# Найдем время обучения и предсказания модели CatBoost
start_time = datetime.now()
best_model_cat = search_cat.best_estimator_
best_model_cat.fit(features_train, target_train)
end_time = datetime.now()
time_cat_fit = end_time - start_time

start_time = datetime.now()
predictions=best_model_cat.predict(features_test)
end_time = datetime.now()
time_cat_predict = end_time - start_time
print('Время обучения модели CatBoost:', time_cat_fit)
print('Время предсказания модели CatBoost:', time_cat_predict)

Learning rate set to 0.177406
0:	learn: 4000.0839490	total: 11.1ms	remaining: 5.56s
1:	learn: 3588.8727232	total: 23ms	remaining: 5.72s
2:	learn: 3271.8316376	total: 34.5ms	remaining: 5.71s
3:	learn: 3029.8388760	total: 45.8ms	remaining: 5.68s
4:	learn: 2838.5385902	total: 57ms	remaining: 5.64s
5:	learn: 2691.5534057	total: 68.4ms	remaining: 5.63s
6:	learn: 2574.1743190	total: 79.6ms	remaining: 5.61s
7:	learn: 2469.9204232	total: 90.9ms	remaining: 5.59s
8:	learn: 2391.9739028	total: 102ms	remaining: 5.56s
9:	learn: 2330.0668724	total: 113ms	remaining: 5.54s
10:	learn: 2274.7322595	total: 124ms	remaining: 5.52s
11:	learn: 2229.4254382	total: 135ms	remaining: 5.5s
12:	learn: 2191.8060942	total: 147ms	remaining: 5.5s
13:	learn: 2159.4647418	total: 158ms	remaining: 5.49s
14:	learn: 2131.4380617	total: 171ms	remaining: 5.54s
15:	learn: 2105.7283992	total: 183ms	remaining: 5.53s
16:	learn: 2084.9648161	total: 195ms	remaining: 5.53s
17:	learn: 2068.1496135	total: 206ms	remaining: 5.51s
18:	le

162:	learn: 1705.3925784	total: 1.78s	remaining: 3.69s
163:	learn: 1704.4709016	total: 1.8s	remaining: 3.68s
164:	learn: 1703.4844068	total: 1.81s	remaining: 3.67s
165:	learn: 1702.9498143	total: 1.82s	remaining: 3.66s
166:	learn: 1702.1700433	total: 1.83s	remaining: 3.65s
167:	learn: 1701.4955745	total: 1.84s	remaining: 3.64s
168:	learn: 1700.9637909	total: 1.85s	remaining: 3.63s
169:	learn: 1700.2644911	total: 1.86s	remaining: 3.62s
170:	learn: 1699.3738811	total: 1.87s	remaining: 3.6s
171:	learn: 1698.6515611	total: 1.88s	remaining: 3.59s
172:	learn: 1697.9840170	total: 1.89s	remaining: 3.58s
173:	learn: 1697.5398967	total: 1.91s	remaining: 3.57s
174:	learn: 1697.0114203	total: 1.92s	remaining: 3.56s
175:	learn: 1696.1632674	total: 1.93s	remaining: 3.55s
176:	learn: 1695.3415980	total: 1.94s	remaining: 3.54s
177:	learn: 1694.9063331	total: 1.95s	remaining: 3.52s
178:	learn: 1694.4232891	total: 1.96s	remaining: 3.51s
179:	learn: 1693.7251748	total: 1.97s	remaining: 3.5s
180:	learn: 1

314:	learn: 1627.3489562	total: 3.43s	remaining: 2.01s
315:	learn: 1627.0363120	total: 3.44s	remaining: 2s
316:	learn: 1626.5856011	total: 3.45s	remaining: 1.99s
317:	learn: 1626.3317005	total: 3.46s	remaining: 1.98s
318:	learn: 1626.1045603	total: 3.47s	remaining: 1.97s
319:	learn: 1625.7834386	total: 3.48s	remaining: 1.96s
320:	learn: 1625.3938703	total: 3.49s	remaining: 1.95s
321:	learn: 1625.1533936	total: 3.51s	remaining: 1.94s
322:	learn: 1625.0001720	total: 3.52s	remaining: 1.93s
323:	learn: 1624.7568506	total: 3.53s	remaining: 1.92s
324:	learn: 1624.3204153	total: 3.54s	remaining: 1.91s
325:	learn: 1623.9146296	total: 3.55s	remaining: 1.9s
326:	learn: 1623.4298682	total: 3.56s	remaining: 1.89s
327:	learn: 1623.1056296	total: 3.58s	remaining: 1.88s
328:	learn: 1622.8480942	total: 3.59s	remaining: 1.86s
329:	learn: 1622.5916260	total: 3.6s	remaining: 1.85s
330:	learn: 1622.3070311	total: 3.61s	remaining: 1.84s
331:	learn: 1621.8229251	total: 3.62s	remaining: 1.83s
332:	learn: 162

481:	learn: 1577.2922012	total: 5.26s	remaining: 196ms
482:	learn: 1577.0439433	total: 5.27s	remaining: 186ms
483:	learn: 1576.8494692	total: 5.28s	remaining: 175ms
484:	learn: 1576.4882023	total: 5.29s	remaining: 164ms
485:	learn: 1576.2506039	total: 5.3s	remaining: 153ms
486:	learn: 1575.9728646	total: 5.32s	remaining: 142ms
487:	learn: 1575.8030354	total: 5.33s	remaining: 131ms
488:	learn: 1575.5450284	total: 5.34s	remaining: 120ms
489:	learn: 1575.3516529	total: 5.35s	remaining: 109ms
490:	learn: 1575.1109710	total: 5.36s	remaining: 98.2ms
491:	learn: 1574.7068710	total: 5.37s	remaining: 87.3ms
492:	learn: 1574.5790355	total: 5.38s	remaining: 76.4ms
493:	learn: 1574.1816022	total: 5.39s	remaining: 65.5ms
494:	learn: 1573.8301426	total: 5.4s	remaining: 54.6ms
495:	learn: 1573.4543669	total: 5.41s	remaining: 43.7ms
496:	learn: 1573.2920941	total: 5.42s	remaining: 32.7ms
497:	learn: 1573.0736422	total: 5.43s	remaining: 21.8ms
498:	learn: 1572.8522128	total: 5.45s	remaining: 10.9ms
499

In [84]:
# Перенесем результаты в единую таблицу
data = [['fit', time_lr_fit, time_lgbm_fit, time_xgb_fit, time_cat_fit],
       ['predict', time_lr_predict, time_lgbm_predict, time_xgb_predict, time_cat_predict]]
df_time = pd.DataFrame(data = data, columns = ['process', 'lin_reg', 'LightGBM', 'XGBoost', 'CatBoost'])
df_time

,process,lin_reg,LightGBM,XGBoost,CatBoost
0,fit,0 days 00:00:02.601249,0 days 00:00:02.104028,0 days 00:01:08.542189,0 days 00:00:05.833271
1,predict,0 days 00:00:00.145788,0 days 00:00:00.335054,0 days 00:00:00.261927,0 days 00:00:00.032638


## Выводы

- В ходе работы мы изучили данные
- Удалили дубликаты в данных; заполнили пропуски в колонках с моделью, типом кузова, типом топлива, информацией о ремонте автомобиля
- Написали функцию для нахождения самого частого типа кузова при определенной марке и модели автомобиля, заполнили пропуски в типах кузова
- Обучили различные модели: линейную регрессию, LightGBM, XGBoost, CatBoost, а так же сравнили скорость обучения моделей
- На наших данных быстрее всего обучается модель LightGBM, и лучшее значение RMSE = 1634 принадлежит этой модели.